<a href="https://colab.research.google.com/github/emoceanographer/usdanutrients/blob/master/Dietary_Sufficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Calculate dietary sufficiency for current diet
(1) get 2011-2013 average of current diet from FBS; include only protein rich foods (kg/capita/yr; protein/cap/day)
(2) use database to grab AA content (cooked) for each food group (10 + bulk) (g/100g)
(3) multiply to get total AA content per food group (g/capita/day)
(4) remove 33%(?) for loss and waste
(5) Get standardized digestibilities for each food
(6) multiply AA amount by digestibility (g/cap/day)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# import necessary packages
import pandas as pd
import re
import statistics as stat
import numpy as np

In [0]:
# defines path to load the poore and nemecek data
path = '/content/drive/My Drive/Colab Notebooks/Nutrition/'
fao_path = path + 'FoodBalanceSheets_E_All_Data.csv' # Poore & Nemecek database tab; values only
nutr_path = path + 'nutrient_output_1126.csv'
dig_path = path + 'digestibility_aa2.csv'
cats_path = path + 'FAO_food_categories.xlsx'
EAT_lancet_path = path + 'eatlancet_ctry_diets.csv'
ctry_path = path + 'fao_ctry_codes.csv'

# (0) Pre-processing

In [0]:
dig_data = pd.read_csv(dig_path)
dig_data = dig_data.replace('-', np.nan)
dig_data = dig_data.replace('Butter ghee', 'Butter, Ghee')
dig_data = dig_data.replace('Meat Aquatic Mammals', 'Meat, Aquatic Mammals')
dig_data = dig_data.replace('Fish Body Oil', 'Fish, Body Oil')
dig_data = dig_data.replace('wheat', 'Wheat and products')

## (1) Gets country diet from FBS
FAO "Food" = Production + Import Quantity - Seed - Losses; food supply is then divided by number of people. "On the utilization side a distinction is made between the quantities exported, fed to livestock, used for seed, put to manufacture for food use and non-food uses, losses during storage and transportation, and food supplies available for human consumption."

In [0]:
# loads the data as a pandas dataframe
fao_data = pd.read_csv(fao_path,encoding = 'latin')

In [0]:
# loads the nutrient data we calculated
nutrient_data = pd.read_csv(nutr_path)

In [0]:
# Fixes items that don't match FBS
nutrient_data = nutrient_data.replace('wheat', 'Wheat and products')
nutrient_data = nutrient_data.replace('Butter ghee', 'Butter, Ghee')
nutrient_data = nutrient_data.replace('Meat Aquatic Mammals', 'Meat, Aquatic Mammals')
nutrient_data = nutrient_data.replace('Fish Body Oil', 'Fish, Body Oil')

In [0]:
ctry_list = list(fao_data.Area.unique())

In [0]:
foods_cat = pd.read_excel(cats_path)
fao_foods = list(foods_cat.Item.unique()) # this is the FAO list

In [0]:
def country_diet(country, temp):
  """Gets the food supply for all foods within a country for 2011-2013"""
  ctry_diet = {}
  df_ctry = fao_data[fao_data['Area']==country]
  # foods to go through (from our nutrient calculations)
  #foods = list(nutrient_data.food.unique()) # goes through the smaller list of foods
  foods = fao_foods
  for item in foods: 
    #temp = df_ctry[(df_ctry['Item'].str.lower()==item.lower())]
    temp = df_ctry[(df_ctry['Item'].str.lower().isin([item.lower()])) & 
                    (df_ctry['Element']=='Food supply quantity (kg/capita/yr)')][['Y2011','Y2012','Y2013']]

    amt = list(pd.DataFrame.mean(temp,axis=1)) # takes the average across years
    if amt:
      ctry_diet[item] = amt[0]
    else:
      ctry_diet[item] = np.nan
  return ctry_diet

In [0]:
# This is in kg/capita/year
ctry_diets = {}
for country in ctry_list: # cycle through each country
  ctry_diets[country] = country_diet(country,fao_foods)


In [0]:
ctry_diet_df = pd.DataFrame.from_dict(ctry_diets,orient='index') # converts it to a dataframe
# for easier viewing

In [13]:
ctry_diet_df.head()

,Wheat and products,Rice (Milled Equivalent),Barley and products,Maize and products,Rye and products,Oats,Millet and products,Sorghum and products,"Cereals, Other",Cassava and products,Potatoes and products,Sweet potatoes,Yams,"Roots, Other",Sugar cane,Sugar beet,Sugar non-centrifugal,Sugar (Raw Equivalent),"Sweeteners, Other",Honey,Beans,Peas,"Pulses, Other and products",Nuts and products,Soyabeans,Groundnuts (Shelled Eq),Sunflower seed,Rape and Mustardseed,Cottonseed,Coconuts - Incl Copra,Sesame seed,Palm kernels,Olives (including preserved),"Oilcrops, Other",Soyabean Oil,Groundnut Oil,Sunflowerseed Oil,Rape and Mustard Oil,Cottonseed Oil,Palmkernel Oil,...,Dates,Grapes and products (excl wine),"Fruits, Other",Coffee and products,Cocoa Beans and products,Tea (including mate),Pepper,Pimento,Cloves,"Spices, Other",Wine,Beer,"Beverages, Fermented","Beverages, Alcoholic","Alcohol, Non-Food",Bovine Meat,Mutton & Goat Meat,Pigmeat,Poultry Meat,"Meat, Other","Offals, Edible","Butter, Ghee",Cream,"Fats, Animals, Raw","Fish, Body Oil","Fish, Liver Oil",Eggs,Milk - Excluding Butter,Freshwater Fish,Demersal Fish,Pelagic Fish,"Marine Fish, Other",Crustaceans,Cephalopods,"Molluscs, Other","Meat, Aquatic Mammals","Aquatic Animals, Others",Aquatic Plants,Infant food,Miscellaneous
Afghanistan,161.116667,14.810000,2.663333,2.540000,NaN,NaN,0.450000,NaN,0.000000,NaN,6.640000,NaN,NaN,NaN,NaN,NaN,NaN,8.326667,0.693333,0.070000,NaN,NaN,2.440000,2.076667,NaN,NaN,NaN,NaN,NaN,0.000000,0.546667,NaN,0.066667,NaN,0.540000,0.000000,0.616667,0.070000,0.11,NaN,...,0.173333,14.376667,7.333333,0.000000,0.020000,3.403333,0.000000,NaN,NaN,0.296667,0.000000,0.090000,NaN,0.000000,NaN,4.853333,5.210000,NaN,2.213333,0.390000,1.813333,1.166667,0.273333,0.520000,NaN,NaN,1.086667,61.123333,0.070000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063333,0.063333
Albania,138.976667,7.110000,0.160000,2.070000,0.676667,0.063333,NaN,NaN,0.073333,0.000000,40.453333,0.813333,NaN,0.00,NaN,NaN,NaN,17.443333,27.986667,0.943333,4.953333,0.000000,0.316667,3.966667,0.003333,0.133333,NaN,0.0,NaN,0.016667,NaN,NaN,20.946667,NaN,0.086667,0.010000,4.946667,0.016667,NaN,NaN,...,3.746667,47.260000,42.286667,2.376667,0.623333,0.090000,0.003333,0.003333,NaN,0.056667,7.666667,27.273333,0.000000,2.680000,NaN,22.046667,14.490000,10.983333,13.133333,0.040000,3.763333,1.120000,0.113333,1.696667,0.0,0.0,10.953333,301.613333,0.693333,0.790000,2.013333,0.730000,0.310000,0.380000,0.310000,NaN,0.0,0.0,0.076667,0.076667
Algeria,185.633333,2.610000,13.333333,15.530000,0.000000,0.003333,NaN,0.053333,0.023333,0.000000,64.936667,NaN,NaN,0.00,NaN,NaN,NaN,27.970000,0.583333,0.156667,1.663333,0.426667,5.330000,1.670000,0.000000,0.240000,NaN,0.0,NaN,0.136667,0.110000,0.0,3.726667,NaN,7.743333,0.473333,2.120000,0.436667,NaN,NaN,...,19.540000,11.636667,27.533333,3.266667,0.916667,0.390000,0.070000,0.480000,0.00,0.270000,0.010000,4.340000,0.000000,0.140000,NaN,5.426667,7.390000,0.000000,7.466667,0.360000,1.400000,0.526667,NaN,0.273333,0.0,0.0,7.316667,146.030000,0.246667,0.420000,2.710000,0.410000,0.113333,0.046667,0.010000,NaN,0.0,0.0,0.543333,0.543333
Angola,39.790000,8.040000,0.000000,38.396667,NaN,0.013333,1.476667,0.000000,NaN,197.060000,22.853333,41.766667,NaN,0.00,NaN,NaN,NaN,14.386667,0.010000,1.113333,8.956667,NaN,NaN,0.100000,0.400000,1.970000,NaN,NaN,NaN,0.170000,0.136667,NaN,NaN,NaN,3.603333,0.736667,0.220000,0.013333,0.03,0.33,...,0.000000,0.096667,2.410000,1.810000,0.166667,0.030000,0.000000,0.000000,0.00,0.013333,4.770000,54.126667,19.100000,1.986667,NaN,7.896667,1.080000,8.406667,16.470000,0.430000,1.120000,0.080000,NaN,0.633333,0.0,NaN,1.720000,11.973333,0.950000,6.313333,7.283333,1.173333,0.040000,0.060000,0.000000,NaN,0.0,0.0,0.193333,0.193333
Antigua and Barbuda,61.260000,9.046667,NaN,2.100000,0.133333,2.150000,NaN,NaN,0.880000,1.296667,15.236667,4.996667,1.343333,1.44,NaN,NaN,NaN,26.370000,4.660000,0.323333,1.420000,0.573333,0.696667,0.680000,0.160000,0.686667,0.05,0.0,NaN,0.580000,0.040000,NaN,0.

#(1.1) EAT-Lancet diet
year = 2010
item = g/d_w
measure = abs %(absolute value)
diet_scn = BMK, FLX, etc.
kcal_scn = 2500 or 2100

translate EAT-Lancet diet into same food groups as ctry_diets

In [0]:
cats_df = pd.read_excel(cats_path)

In [18]:
# cycles through each EAT-Lancet category and assigns amount eaten 
# to those categories
fao_el_trans = {}
for item in cats_df['EAT-Lancet cats'].unique():
  temp = cats_df[cats_df['EAT-Lancet cats']==item]
  if len(temp)>0:
    fao_el_trans[item] = list(temp['Item'])
  else:
    print([item, 'not present in EAT-Lancet categories'])


[nan, 'not present in EAT-Lancet categories']


In [0]:
eld = pd.read_csv(EAT_lancet_path)
# trims the data to the years / scenarios we are interested in
eld = eld[(eld['measure']=='abs') & (eld['year']==2010) & (eld['item']=='g/d_w') & (eld['kcal_scn'] == '2100kcal')]

In [27]:
eld['value'] = eld['value']/1000*365 # g/d to kg/yr
eld.head()

,kcal_scn,SSP_scn,measure,item,diet_scn,food_group,region,year,value
0,2100kcal,SSP2,abs,g/d_w,BMK,wheat,all-r,2010,42.933934
3,2100kcal,SSP2,abs,g/d_w,BMK,wheat,HIC,2010,49.553797
6,2100kcal,SSP2,abs,g/d_w,BMK,wheat,UMC,2010,56.504974
9,2100kcal,SSP2,abs,g/d_w,BMK,wheat,LMC,2010,44.189742
12,2100kcal,SSP2,abs,g/d_w,BMK,wheat,LIC,2010,20.659071


In [28]:
# Run without apportioning foods from FBS
ctry_diet_el = {}
for country in ['AFG']:#eld['region'].unique(): 
  print(country)
  ctry_diet_el[country] = {}
  for item in fao_el_trans:
    no_fao_items = len(fao_el_trans[item])
    for fao_item in fao_el_trans[item]:
      temp = eld[(eld['region']==country) & (eld['food_group'] == item) & (eld['diet_scn']=='FLX')]
      if len(temp)>0:
        ctry_diet_el[country][fao_item] = temp['value'].values[0]/no_fao_items
      else:
        ctry_diet_el[country][fao_item] = np.nan

AFG


In [30]:
ctry_diet_el['AFG']

{'Apples and products': 6.636363636363637,
 'Aquatic Animals, Others': 1.1355555555555557,
 'Aquatic Plants': nan,
 'Bananas': 6.636363636363637,
 'Barley and products': 0.3160724798783333,
 'Beans': 6.083333333333333,
 'Bovine Meat': 2.8629428802599994,
 'Cassava and products': 1.16079528252,
 'Cephalopods': 1.1355555555555557,
 'Cereals, Other': 0.3160724798783333,
 'Citrus, Other': 6.636363636363637,
 'Coconut Oil': 1.2166666666666666,
 'Coconuts - Incl Copra': 1.825,
 'Cottonseed': 1.825,
 'Cottonseed Oil': 1.2166666666666666,
 'Crustaceans': 1.1355555555555557,
 'Dates': 6.636363636363637,
 'Demersal Fish': 1.1355555555555557,
 'Eggs': 0.673920798845,
 'Freshwater Fish': 1.1355555555555557,
 'Fruits, Other': 6.636363636363637,
 'Grapefruit and products': 6.636363636363637,
 'Grapes and products (excl wine)': 6.636363636363637,
 'Groundnut Oil': 1.2166666666666666,
 'Groundnuts (Shelled Eq)': 1.825,
 'Honey': 0.9177447951083334,
 'Lemons, Limes and products': 6.636363636363637,
 'M

In [21]:
ctry_diet_el

{'AFG': {'Apples and products': 18.181818181818183,
  'Aquatic Animals, Others': 3.111111111111111,
  'Aquatic Plants': nan,
  'Bananas': 18.181818181818183,
  'Barley and products': 0.8659519996666667,
  'Beans': 16.666666666666668,
  'Bovine Meat': 7.8436791239999994,
  'Cassava and products': 3.1802610479999998,
  'Cephalopods': 3.111111111111111,
  'Cereals, Other': 0.8659519996666667,
  'Citrus, Other': 18.181818181818183,
  'Coconut Oil': 3.3333333333333335,
  'Coconuts - Incl Copra': 5.0,
  'Cottonseed': 5.0,
  'Cottonseed Oil': 3.3333333333333335,
  'Crustaceans': 3.111111111111111,
  'Dates': 18.181818181818183,
  'Demersal Fish': 3.111111111111111,
  'Eggs': 1.846358353,
  'Freshwater Fish': 3.111111111111111,
  'Fruits, Other': 18.181818181818183,
  'Grapefruit and products': 18.181818181818183,
  'Grapes and products (excl wine)': 18.181818181818183,
  'Groundnut Oil': 3.3333333333333335,
  'Groundnuts (Shelled Eq)': 5.0,
  'Honey': 2.5143693016666666,
  'Lemons, Limes and 

In [0]:
ctry_diet_el_df = pd.DataFrame.from_dict(ctry_diet_el,orient='index')

## (2) Use database to get cooked food amino acid content for each food

In [0]:
nutrient_data.head()

,Unnamed: 0,food,status,avg/std,Tryptophan,Threonine,Isoleucine,Leucine,Lysine,Methionine,Cystine,Phenylalanine,Tyrosine,Valine,Arginine,Histidine,Alanine,Aspartic,Glutamic,Glycine,Serine,Hydroxyproline,energy,B12,B6,protein,iron
0,0,Barley and products,raw,avg,0.170000,0.381000,0.390500,0.745000,0.440000,0.231500,0.221000,0.517500,0.321000,0.529000,0.620750,0.253750,0.449250,0.707250,2.603750,0.407750,0.463250,NaN,357.600000,0.714000,0.468600,11.274000,12.340000
1,1,Barley and products,raw,std,0.031294,0.041174,0.044829,0.074873,0.075481,0.046801,0.049146,0.204497,0.034341,0.056598,0.153721,0.028605,0.061716,0.082447,0.593721,0.045258,0.046636,NaN,11.760102,1.596553,0.203980,1.467270,20.071140
2,2,Barley and products,cooked,avg,0.038000,0.077000,0.083000,0.154000,0.084000,0.043000,0.050000,0.127000,0.065000,0.111000,0.113000,0.051000,0.088000,0.141000,0.591000,0.082000,0.095000,NaN,123.000000,0.000000,0.115000,2.260000,1.330000
3,3,Barley and products,cooked,std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Beans,raw,avg,0.193561,0.618293,0.713048,1.233690,1.043595,0.205738,0.173619,0.824929,0.466927,0.810810,0.989829,0.438714,0.677125,1.893925,2.381550,0.624650,0.842100,0.0,234.644444,0.000000,0.265289,15.241333,4.816222


In [0]:
fruits_veg = ['Tomatoes and products', 'Onions', 'Vegetables, Other', 'Oranges, Mandarines',
             'Lemons, Limes and products', 'Grapefruit and products', 'Citrus, other', 
             'Bananas', 'Plantains', 'Apples and products', 'Dates', 
              'Grapes and products (excl wine)','Fruits, Other','Pimento']
# these should be used as a mix of raw AND cooked

In [0]:
nutrient_means = nutrient_data[(nutrient_data['status']=='cooked') & 
                               (nutrient_data['avg/std']=='avg')] # gets only 

In [0]:
num_cols = ['Tryptophan', 'Threonine',
       'Isoleucine', 'Leucine', 'Lysine', 'Methionine', 'Cystine',
       'Phenylalanine', 'Tyrosine', 'Valine', 'Arginine', 'Histidine',
       'Alanine', 'Aspartic', 'Glutamic', 'Glycine', 'Serine',
       'Hydroxyproline', 'energy', 'B12', 'B6', 'protein', 'iron']

In [0]:
code2_foods = {'Aquatic Animals, Others': ['Meat, Aquatic Mammals', 'Crustaceans', 
                                           'Cephalopods', 'Pelagic Fish', 'Demersal Fish',
                                           'Freshwater Fish'], 
               'Cereals, Other': ['Wheat and products', 'Rice (Milled Equivalent)', 
                                 'Barley and products', 'Maize and products', 
                                  'Rye and products','Oats', 'Millet and products', 
                                 'Sorghum and products'],
               'Citrus, other': ['Oranges, Mandarines', 'Lemons, Limes and products',
                                 'Grapefruit and products'],
               'Fish, Body Oil': ['Pelagic Fish', 'Demersal Fish', 'Freshwater Fish'],
               'Fish, Liver Oil': ['Pelagic Fish', 'Demersal Fish', 'Freshwater Fish'],
               'Fruits, Other': ['Bananas', 'Plantains', 'Apples and products', 'Dates',
                                'Grapes and products (excl wine)'], 
               'Marine Fish, Other': ['Pelagic Fish', 'Demersal Fish'], 
               'Vegetables, Other': ['Tomatoes and products', 'Onions'],
              'Roots, Other': ['Potatoes and products', 'Sweet potatoes'],
              'Meat, Other': ['Poultry Meat', 'Bovine Meat', 'Mutton & Goat Meat',
                             'Pigmeat'],
              'Offals, Edible': ['Poultry Meat', 'Bovine Meat', 'Mutton & Goat Meat',
                             'Pigmeat'],
              'Pelagic Fish': ['Demersal Fish'], 
              'Molluscs, Other': ['Crustaceans'],
              'Pulses, Other and products': ['Soyabeans', 'Beans', 'Peas']}


In [0]:
for item in list(code2_foods.keys()):
  if not item in fruits_veg:
    item_mix = nutrient_data[(nutrient_data['food'].isin(code2_foods[item])) & 
                             (nutrient_data['status']=='raw') &
                             (nutrient_data['avg/std'] == 'avg')]
  else: 
    item_mix = nutrient_data[(nutrient_data['food'].isin(code2_foods[item])) &
                        (nutrient_data['avg/std'] == 'avg')]
  avg_val = item_mix[num_cols].mean(axis=0)
  avg_val['Unnamed: 0'] = item_mix[item_mix['status']=='raw']['Unnamed: 0'].values[0]
  avg_val['food'] = item
  avg_val['status'] = 'both'
  avg_val['avg/std'] = 'avg'
  idx = nutrient_means.index[nutrient_means['food']==item]

  nutrient_means = nutrient_means.drop(idx)
  nutrient_means = nutrient_means.append(avg_val,ignore_index=True)

In [0]:
milk_raw = nutrient_data[(nutrient_data['food']=='Milk - Excluding Butter') &
                        (nutrient_data['status']=='raw') &
                        (nutrient_data['avg/std'] == 'avg')]
idx = nutrient_means.index[nutrient_means['food']=='Milk - Excluding Butter']

nutrient_means = nutrient_means.drop(idx)
nutrient_means = nutrient_means.append(milk_raw)

for item in fruits_veg: 
  if not item in list(code2_foods.keys()): # already caught the code2 veggies
    veg_rawcook = nutrient_data[(nutrient_data['food']==item) &
                        (nutrient_data['avg/std'] == 'avg')]
 
    avg_val = veg_rawcook[num_cols].mean(axis=0)
    avg_val['Unnamed: 0'] = veg_rawcook[veg_rawcook['status']=='raw']['Unnamed: 0'].values[0]
    avg_val['food'] = item
    avg_val['status'] = 'both'
    avg_val['avg/std'] = 'avg'
    idx = nutrient_means.index[nutrient_means['food']==item]

    nutrient_means = nutrient_means.drop(idx)
    nutrient_means = nutrient_means.append(avg_val,ignore_index=True)

# these are in g/100g

In [0]:
aa_list = {'Tryptophan': 'TRP', 'Threonine':'THR', 'Isoleucine':'ILE',
           'Leucine':'LEU', 'Lysine':'LYS','Methionine':'MET',
           'Cystine':'CYS', 'Phenylalanine':'PHE', 'Tyrosine':'TYR',
           'Valine':'VAL','Arginine':'ARG', 'Histidine':'HIS', 
           'Alanine':'ALA', 'Aspartic':'ASP', 'Glutamic':'GLU', 
           'Glycine':'GLY', 'Serine':'SER','protein':'CP'}
# ignoring 'Hydroxyproline':'HYP', 'Proline':'PRO' because of lack of data

## (3) Get digestibilities for each food

In [0]:
# calculate digestible AA per food
import math

digestible_aa = {}

for food in fao_foods:
  temp_nut = nutrient_means[nutrient_means['food'].str.lower().isin([food.lower()])]
  temp_dig = dig_data[dig_data['Unnamed: 0'].str.lower().isin([food.lower()])]

  if not len(temp_nut) >0: # skips those like sugar
    print(food)
  else:
    food_dig_aa = {}
    for AA in aa_list: 
      #if food == 'Butter, Ghee':
       # print(temp_dig[aa_list[AA]])
        #if temp_dig[aa_list[AA]].empty:
         # print('yes')
      if math.isnan(temp_nut[AA]):
          aa_content = np.nan
      else:
        aa_content = float(temp_nut[AA])

      if temp_dig[aa_list[AA]].empty:
        aa_dig = np.nan
      else:
        aa_dig = float(temp_dig[aa_list[AA]])
      dig_aa_content = aa_dig * aa_content / 100 # aa_dig is a percent

      food_dig_aa[AA] = dig_aa_content

    digestible_aa[food] = food_dig_aa
  

Sugar cane
Sugar beet
Sugar non-centrifugal
Sugar (Raw Equivalent)
Sweeteners, Other
Rape and Mustardseed
Olives (including preserved)
Oilcrops, Other
Groundnut Oil
Palmkernel Oil
Palm Oil
Sesameseed Oil
Ricebran Oil
Maize Germ Oil
Oilcrops Oil, Other
Tea (including mate)
Pepper
Cloves
Spices, Other
Wine
Beverages, Fermented
Beverages, Alcoholic
Alcohol, Non-Food
Fats, Animals, Raw
Infant food
Miscellaneous


In [0]:
dig_aa_df = pd.DataFrame.from_dict(digestible_aa)
dig_aa_df.head()

,Wheat and products,Rice (Milled Equivalent),Barley and products,Maize and products,Rye and products,Oats,Millet and products,Sorghum and products,"Cereals, Other",Cassava and products,Potatoes and products,Sweet potatoes,Yams,"Roots, Other",Honey,Beans,Peas,"Pulses, Other and products",Nuts and products,Soyabeans,Groundnuts (Shelled Eq),Sunflower seed,Cottonseed,Coconuts - Incl Copra,Sesame seed,Palm kernels,Soyabean Oil,Sunflowerseed Oil,Rape and Mustard Oil,Cottonseed Oil,Coconut Oil,Olive Oil,Tomatoes and products,Onions,"Vegetables, Other","Oranges, Mandarines","Lemons, Limes and products",Grapefruit and products,"Citrus, Other",Bananas,Plantains,Apples and products,Pineapples and products,Dates,Grapes and products (excl wine),"Fruits, Other",Coffee and products,Cocoa Beans and products,Pimento,Beer,Bovine Meat,Mutton & Goat Meat,Pigmeat,Poultry Meat,"Meat, Other","Offals, Edible","Butter, Ghee",Cream,"Fish, Body Oil","Fish, Liver Oil",Eggs,Milk - Excluding Butter,Freshwater Fish,Demersal Fish,Pelagic Fish,"Marine Fish, Other",Crustaceans,Cephalopods,"Molluscs, Other","Meat, Aquatic Mammals","Aquatic Animals, Others",Aquatic Plants
Tryptophan,0.076263,0.036252,0.023053,0.016111,NaN,0.074108,0.033440,NaN,0.096425,NaN,0.010828,0.006534,NaN,0.008454,NaN,0.042016,0.026565,0.097677,0.135382,0.347119,0.195621,0.233809,0.392370,0.020728,0.304357,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.003990,0.005141,0.004566,0.001504,0.000675,0.001761,0.001314,0.004860,0.004185,0.001086,0.001395,NaN,0.001215,0.002634,NaN,NaN,0.006793,NaN,0.241707,0.266043,0.236628,0.241098,0.189746,0.189746,NaN,0.070479,0.185364,0.185364,NaN,0.041089,0.221442,0.239451,0.167734,0.167734,0.212310,0.252750,NaN,0.582590,0.180937,NaN
Threonine,0.164227,0.090120,0.044403,0.082161,NaN,0.140237,0.078535,NaN,0.214884,NaN,0.029984,0.009940,NaN,0.011708,NaN,0.148332,0.125132,0.340357,0.364233,0.975548,0.563555,0.587088,0.864588,0.065897,0.541125,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.006879,0.005184,0.006031,0.001871,0.000260,0.001436,0.001375,0.009848,0.004745,0.001416,0.001322,NaN,0.002226,0.004656,NaN,NaN,0.009460,NaN,1.124655,1.048322,0.963469,0.906813,0.739732,0.739732,NaN,0.223028,0.704437,0.704437,1.752738,0.135813,0.925499,0.926834,0.627487,0.627487,0.672586,0.987280,NaN,2.296420,0.674260,NaN
Isoleucine,0.224831,0.123353,0.052567,0.106071,NaN,0.217464,0.119880,NaN,0.282161,NaN,0.036981,0.017834,NaN,0.023240,NaN,0.181676,0.150062,0.426279,0.467099,1.204184,0.688412,0.756160,0.898110,0.074399,0.634808,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.012759,0.013207,0.012983,0.004770,0.000560,0.001867,0.002767,0.020790,0.008680,0.003267,0.002847,NaN,0.001890,0.009217,NaN,NaN,0.017076,NaN,1.123778,1.051501,0.958381,0.954329,0.728699,0.728699,NaN,0.294677,0.750965,0.750965,1.943885,0.167951,0.953535,1.002839,0.677371,0.677371,0.806272,1.035960,NaN,2.119312,0.725747,NaN
Leucine,0.429414,0.246462,0.101640,0.330593,NaN,0.440944,0.388020,NaN,0.729428,NaN,0.061009,0.034704,NaN,0.044516,NaN,0.319410,0.266062,0.731958,0.866880,1.958650,1.382987,1.101497,1.702890,0.148853,1.166733,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.023747,0.020213,0.021980,0.007800,0.005600,0.004242,0.005937,0.057138,0.021700,0.007088,0.005425,NaN,0.004987,0.023647,NaN,NaN,0.034496,NaN,2.023717,1.746907,1.678769,1.608958,1.273552,1.273552,NaN,0.554241,1.305442,1.305442,3.089477,0.317205,1.644031,1.766078,1.188716,1.188716,1.363548,1.677057,NaN,4.386810,1.283008,NaN
Lysine,0.193784,0.095995,0.052080,0.100312,NaN,0.260246,0.043550,NaN,0.216357,NaN,0.061320,0.024948,NaN,0.044261,NaN,0.298405,0.269667,0.672198,0.413044,1.644013,0.648546,0.570078,1.116450,0.085876,0.451748,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.022435,0.022206,0.022321,0.011267,0.004950,0.005209,0.007142,0.030442,0.020460,0.008790,0.006215,NaN,0.005115,0.016119,NaN,NaN,0.028222,NaN,2.350750,2.091732,1.939890,1.930503,1.512047,1.512047,NaN,0.456560,1.487252,1.487252,2.437025,0.268892,1.857200,2.005615,1.354337,1.354337,1.479659,1.769617,NaN,5.019600,1.405513,NaN


##(4) take into account food loss and waste 

In [0]:
# WRI data; less specific?
food_loss_waste = {'North America': {'Consumption': 61, 'Dist & Mkt': 7, 'Processing': 9,
                                     'Handling & Storage': 6, 'Production': 17},
                  'Industrialized Asia': {'Consumption': 46, 'Dist & Mkt': 11, 'Processing': 2,
                                     'Handling & Storage': 23, 'Production': 17},
                  'Europe': {'Consumption': 52, 'Dist & Mkt': 9, 'Processing': 5,
                                     'Handling & Storage': 12, 'Production': 23},
                  'North Africa, West and Central Asia': {'Consumption': 34, 'Dist & Mkt': 18, 'Processing': 4,
                                     'Handling & Storage': 21, 'Production': 23},
                  'Latin America': {'Consumption': 28, 'Dist & Mkt': 17, 'Processing': 6,
                                     'Handling & Storage': 22, 'Production': 28},
                  'South and Southeast Asia': {'Consumption': 13, 'Dist & Mkt': 15, 'Processing': 4,
                                     'Handling & Storage': 37, 'Production': 32},
                  'Sub-Saharan Africa': {'Consumption': 5, 'Dist & Mkt': 13, 'Processing': 7,
                                     'Handling & Storage': 37, 'Production': 39}}
# from Monica's presentation (originally WRI?); percent of total waste
# LATER: compare the amount of LOSSES for each commodity and assume this corresponds
# to whatever percentage corresponds and then add Processing, dist & mkt, consumption?

#### (1) Assign each country in our ctry_diet to a region, (2) group foods into the Gustavsson loss categories to assign loss percentages, and (3) multiply food/country pairs by the country/commodity specific loss amount (processing-consumption only)

In [0]:
# loads data from Gustavsson (2011); country groupings and then the waste per food category for
# countries within that region
ctry_grps = pd.read_excel(path + 'gustavsson_data.xlsx',sheet_name = 'Country_keys')
NAO = pd.read_excel(path + 'gustavsson_data.xlsx',sheet_name = 'N. America and Oceania')
IA = pd.read_excel(path + 'gustavsson_data.xlsx',sheet_name = 'Indust Asia')
SSA = pd.read_excel(path + 'gustavsson_data.xlsx',sheet_name = 'SSA')
NAWCA = pd.read_excel(path + 'gustavsson_data.xlsx',sheet_name = 'N Afr, West and Cent Asia')
Eur = pd.read_excel(path + 'gustavsson_data.xlsx',sheet_name = 'Europe (incl Russia)')
SSEA = pd.read_excel(path + 'gustavsson_data.xlsx',sheet_name = 'S and SE Asia')
LA = pd.read_excel(path + 'gustavsson_data.xlsx',sheet_name = 'L America')


In [0]:
extra_ctries = pd.read_csv(path + 'gustav_ctry_xtra.csv')
extra_ctries.head()

,Country,Region
0,Antigua and Barbuda,LA
1,Bahamas,LA
2,Barbados,LA
3,Belgium-Luxembourg,exclude
4,Bermuda,exclude


In [0]:
ctry_grps_dict = ctry_grps.to_dict('records')

for element in ctry_grps_dict:
  if element['Area'] == 'Europe':
    element['xtras'] = list(extra_ctries[extra_ctries['Region']=='Eur']['Country'])
    element['var'] = Eur
  elif element['Area'] == 'N. America & Oceania':
    element['xtras'] = list(extra_ctries[extra_ctries['Region']=='NAO']['Country'])
    element['var'] = NAO
  elif element['Area'] == 'SSA':
    element['xtras'] = list(extra_ctries[extra_ctries['Region']=='SSA']['Country'])
    element['var'] = SSA
  elif element['Area'] == 'N. Africa, W & C Asia':
    element['xtras'] = list(extra_ctries[extra_ctries['Region']=='NAWCA']['Country'])
    element['var'] = NAWCA
  elif element['Area'] == 'Ind. Asia':
    element['xtras'] = list(extra_ctries[extra_ctries['Region']=='IA']['Country'])
    element['var'] = IA
  elif element['Area'] == 'S and SE Asia':
    element['xtras'] = list(extra_ctries[extra_ctries['Region']=='SSEA']['Country'])
    element['var'] = SSEA
  elif element['Area'] == 'LA':
    element['xtras'] = list(extra_ctries[extra_ctries['Region']=='LA']['Country'])
    element['var'] = LA

In [0]:
excludes = list(extra_ctries[extra_ctries['Region']=='exclude']['Country'])

In [0]:
cats_dict = {}
for foods in list(foods_cat['Gustavsson cats'].unique()):
  cats_dict[foods] = list(foods_cat[foods_cat['Gustavsson cats'] == foods]['Item'].unique())
cats_dict['Fruits and vegetables'] = cats_dict.pop('Fruit and vegetable')
cats_dict['Milk'] = cats_dict.pop('Dairy ')

In [0]:
  ctry_diet_df2 = ctry_diet_df.reset_index()
  ctry_diet_df2['waste?'] = 0

In [0]:
for country in list(ctry_diet_df2['index'].unique()):

  ctry_only = ctry_diet_df2[ctry_diet_df2['index'] == country]
  
  ctr = 0
  for element in ctry_grps_dict: 
    if country in element['Countries'] or country in element['xtras']: # find which region it is associated with
      temp_df = element['var']
      temp_df['extra_waste'] = temp_df[['processing and packaging',
                                        'distribution (supermarket retail)', 'Consumption']].sum(axis=1)
      # add a row that assigns a waste amount to each food by category
      for food in list(temp_df['Unnamed: 0'].unique()):
        cat_foods = cats_dict[food]
        unwasted_amt = (1 - temp_df[temp_df['Unnamed: 0']==food]['extra_waste'].values[0]/100)
        ctry_only[cat_foods] = ctry_only[cat_foods]*unwasted_amt
      ctry_only['waste?'] = 1
      
      idx = ctry_diet_df2.index[ctry_diet_df2['index']==country]
      
      ctry_diet_df2 = ctry_diet_df2.drop(idx)
      ctry_diet_df2 = ctry_diet_df2.append(ctry_only,ignore_index=True)
      ctr = 1
  if ctr == 0:
    print(country)
    

    

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Belgium-Luxembourg
Bermuda
French Polynesia
Netherlands Antilles (former)
New Caledonia
The former Yugoslav Republic of Macedonia
Trinidad and Tobago
World
Eastern Africa
Middle Africa
Northern Africa
Southern Africa
Western Africa
Americas
Northern America
Central America
Caribbean
South America
Asia
Central Asia
Eastern Asia
Southern Asia
South-Eastern Asia
Western Asia
Europe
Eastern Europe
Northern Europe
Southern Europe
Western Europe
Oceania
Australia & New Zealand
Melanesia
Micronesia
Polynesia
European Union
Least Developed Countries
Land Locked Developing Countries
Small Island Developing States
Low Income Food Deficit Countries
Net Food Importing Developing Countries


In [0]:
ctry_diet_df2[ctry_diet_df2['index']=='Afghanistan']

,index,Wheat and products,Rice (Milled Equivalent),Barley and products,Maize and products,Rye and products,Oats,Millet and products,Sorghum and products,"Cereals, Other",Cassava and products,Potatoes and products,Sweet potatoes,Yams,"Roots, Other",Sugar cane,Sugar beet,Sugar non-centrifugal,Sugar (Raw Equivalent),"Sweeteners, Other",Honey,Beans,Peas,"Pulses, Other and products",Nuts and products,Soyabeans,Groundnuts (Shelled Eq),Sunflower seed,Rape and Mustardseed,Cottonseed,Coconuts - Incl Copra,Sesame seed,Palm kernels,Olives (including preserved),"Oilcrops, Other",Soyabean Oil,Groundnut Oil,Sunflowerseed Oil,Rape and Mustard Oil,Cottonseed Oil,...,Grapes and products (excl wine),"Fruits, Other",Coffee and products,Cocoa Beans and products,Tea (including mate),Pepper,Pimento,Cloves,"Spices, Other",Wine,Beer,"Beverages, Fermented","Beverages, Alcoholic","Alcohol, Non-Food",Bovine Meat,Mutton & Goat Meat,Pigmeat,Poultry Meat,"Meat, Other","Offals, Edible","Butter, Ghee",Cream,"Fats, Animals, Raw","Fish, Body Oil","Fish, Liver Oil",Eggs,Milk - Excluding Butter,Freshwater Fish,Demersal Fish,Pelagic Fish,"Marine Fish, Other",Crustaceans,Cephalopods,"Molluscs, Other","Meat, Aquatic Mammals","Aquatic Animals, Others",Aquatic Plants,Infant food,Miscellaneous,waste?
40,Afghanistan,147.42175,13.55115,2.43695,2.3241,NaN,NaN,0.41175,NaN,0.0,NaN,5.0464,NaN,NaN,NaN,NaN,NaN,NaN,8.326667,0.693333,0.07,NaN,NaN,2.1716,1.848233,NaN,NaN,NaN,NaN,NaN,0.0,0.486533,NaN,0.059333,NaN,0.54,0.0,0.616667,0.07,0.11,...,8.338467,4.253333,0.0,0.02,3.403333,0.0,NaN,NaN,0.296667,0.0,0.09,NaN,0.0,NaN,4.0768,4.3764,NaN,1.8592,0.3276,1.5232,1.015,0.2378,0.4368,NaN,NaN,0.9454,53.1773,0.0518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063333,0.063333,1


In [0]:
ctry_diet_df.head()

,Wheat and products,Rice (Milled Equivalent),Barley and products,Maize and products,Rye and products,Oats,Millet and products,Sorghum and products,"Cereals, Other",Cassava and products,Potatoes and products,Sweet potatoes,Yams,"Roots, Other",Sugar cane,Sugar beet,Sugar non-centrifugal,Sugar (Raw Equivalent),"Sweeteners, Other",Honey,Beans,Peas,"Pulses, Other and products",Nuts and products,Soyabeans,Groundnuts (Shelled Eq),Sunflower seed,Rape and Mustardseed,Cottonseed,Coconuts - Incl Copra,Sesame seed,Palm kernels,Olives (including preserved),"Oilcrops, Other",Soyabean Oil,Groundnut Oil,Sunflowerseed Oil,Rape and Mustard Oil,Cottonseed Oil,Palmkernel Oil,...,Dates,Grapes and products (excl wine),"Fruits, Other",Coffee and products,Cocoa Beans and products,Tea (including mate),Pepper,Pimento,Cloves,"Spices, Other",Wine,Beer,"Beverages, Fermented","Beverages, Alcoholic","Alcohol, Non-Food",Bovine Meat,Mutton & Goat Meat,Pigmeat,Poultry Meat,"Meat, Other","Offals, Edible","Butter, Ghee",Cream,"Fats, Animals, Raw","Fish, Body Oil","Fish, Liver Oil",Eggs,Milk - Excluding Butter,Freshwater Fish,Demersal Fish,Pelagic Fish,"Marine Fish, Other",Crustaceans,Cephalopods,"Molluscs, Other","Meat, Aquatic Mammals","Aquatic Animals, Others",Aquatic Plants,Infant food,Miscellaneous
Afghanistan,161.116667,14.810000,2.663333,2.540000,NaN,NaN,0.450000,NaN,0.000000,NaN,6.640000,NaN,NaN,NaN,NaN,NaN,NaN,8.326667,0.693333,0.070000,NaN,NaN,2.440000,2.076667,NaN,NaN,NaN,NaN,NaN,0.000000,0.546667,NaN,0.066667,NaN,0.540000,0.000000,0.616667,0.070000,0.11,NaN,...,0.173333,14.376667,7.333333,0.000000,0.020000,3.403333,0.000000,NaN,NaN,0.296667,0.000000,0.090000,NaN,0.000000,NaN,4.853333,5.210000,NaN,2.213333,0.390000,1.813333,1.166667,0.273333,0.520000,NaN,NaN,1.086667,61.123333,0.070000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063333,0.063333
Albania,138.976667,7.110000,0.160000,2.070000,0.676667,0.063333,NaN,NaN,0.073333,0.000000,40.453333,0.813333,NaN,0.00,NaN,NaN,NaN,17.443333,27.986667,0.943333,4.953333,0.000000,0.316667,3.966667,0.003333,0.133333,NaN,0.0,NaN,0.016667,NaN,NaN,20.946667,NaN,0.086667,0.010000,4.946667,0.016667,NaN,NaN,...,3.746667,47.260000,42.286667,2.376667,0.623333,0.090000,0.003333,0.003333,NaN,0.056667,7.666667,27.273333,0.000000,2.680000,NaN,22.046667,14.490000,10.983333,13.133333,0.040000,3.763333,1.120000,0.113333,1.696667,0.0,0.0,10.953333,301.613333,0.693333,0.790000,2.013333,0.730000,0.310000,0.380000,0.310000,NaN,0.0,0.0,0.076667,0.076667
Algeria,185.633333,2.610000,13.333333,15.530000,0.000000,0.003333,NaN,0.053333,0.023333,0.000000,64.936667,NaN,NaN,0.00,NaN,NaN,NaN,27.970000,0.583333,0.156667,1.663333,0.426667,5.330000,1.670000,0.000000,0.240000,NaN,0.0,NaN,0.136667,0.110000,0.0,3.726667,NaN,7.743333,0.473333,2.120000,0.436667,NaN,NaN,...,19.540000,11.636667,27.533333,3.266667,0.916667,0.390000,0.070000,0.480000,0.00,0.270000,0.010000,4.340000,0.000000,0.140000,NaN,5.426667,7.390000,0.000000,7.466667,0.360000,1.400000,0.526667,NaN,0.273333,0.0,0.0,7.316667,146.030000,0.246667,0.420000,2.710000,0.410000,0.113333,0.046667,0.010000,NaN,0.0,0.0,0.543333,0.543333
Angola,39.790000,8.040000,0.000000,38.396667,NaN,0.013333,1.476667,0.000000,NaN,197.060000,22.853333,41.766667,NaN,0.00,NaN,NaN,NaN,14.386667,0.010000,1.113333,8.956667,NaN,NaN,0.100000,0.400000,1.970000,NaN,NaN,NaN,0.170000,0.136667,NaN,NaN,NaN,3.603333,0.736667,0.220000,0.013333,0.03,0.33,...,0.000000,0.096667,2.410000,1.810000,0.166667,0.030000,0.000000,0.000000,0.00,0.013333,4.770000,54.126667,19.100000,1.986667,NaN,7.896667,1.080000,8.406667,16.470000,0.430000,1.120000,0.080000,NaN,0.633333,0.0,NaN,1.720000,11.973333,0.950000,6.313333,7.283333,1.173333,0.040000,0.060000,0.000000,NaN,0.0,0.0,0.193333,0.193333
Antigua and Barbuda,61.260000,9.046667,NaN,2.100000,0.133333,2.150000,NaN,NaN,0.880000,1.296667,15.236667,4.996667,1.343333,1.44,NaN,NaN,NaN,26.370000,4.660000,0.323333,1.420000,0.573333,0.696667,0.680000,0.160000,0.686667,0.05,0.0,NaN,0.580000,0.040000,NaN,0.

## (5) Multiply food amount by the amino acid content per amount to get the amino acid amount per food 

In [0]:
ctry_diet_el_df = ctry_diet_el_df.reset_index()


,level_0,index,Wheat and products,Rice (Milled Equivalent),Barley and products,Rye and products,Oats,Millet and products,Sorghum and products,"Cereals, Other",Maize and products,Cassava and products,Potatoes and products,Sweet potatoes,Yams,"Roots, Other",Sugar cane,Sugar beet,Sugar non-centrifugal,Sugar (Raw Equivalent),"Sweeteners, Other",Honey,Beans,Peas,"Pulses, Other and products",Nuts and products,Groundnuts (Shelled Eq),Sunflower seed,Rape and Mustardseed,Cottonseed,Coconuts - Incl Copra,Sesame seed,Palm kernels,Olives (including preserved),"Oilcrops, Other",Soyabeans,Soyabean Oil,Groundnut Oil,Sunflowerseed Oil,Rape and Mustard Oil,...,Palmkernel Oil,Coconut Oil,Sesameseed Oil,Olive Oil,Ricebran Oil,Maize Germ Oil,"Oilcrops Oil, Other",Palm Oil,Tomatoes and products,Onions,"Vegetables, Other",Pimento,"Oranges, Mandarines","Lemons, Limes and products",Grapefruit and products,"Citrus, Other",Bananas,Plantains,Apples and products,Pineapples and products,Dates,Grapes and products (excl wine),"Fruits, Other",Bovine Meat,Mutton & Goat Meat,Pigmeat,Poultry Meat,Eggs,Milk - Excluding Butter,Freshwater Fish,Demersal Fish,Pelagic Fish,"Marine Fish, Other",Crustaceans,Cephalopods,"Molluscs, Other","Meat, Aquatic Mammals","Aquatic Animals, Others",Aquatic Plants,Miscellaneous
5,5,AFG,162.517467,27.549035,0.865952,0.865952,0.865952,0.865952,0.865952,0.865952,6.333265,3.180261,3.180261,3.180261,3.180261,3.180261,2.514369,2.514369,2.514369,2.514369,2.514369,2.514369,16.666667,16.666667,16.666667,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,25.0,3.333333,3.333333,3.333333,3.333333,...,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,6.8,75.0,75.0,75.0,75.0,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,7.843679,6.156321,NaN,5.314043,1.846358,172.841561,3.111111,3.111111,3.111111,3.111111,3.111111,3.111111,3.111111,3.111111,3.111111,NaN,NaN


In [0]:
ctry_diet_el_df[ctry_diet_el_df['index']=='AFG']

,level_0,index,Wheat and products,Rice (Milled Equivalent),Barley and products,Rye and products,Oats,Millet and products,Sorghum and products,"Cereals, Other",Maize and products,Cassava and products,Potatoes and products,Sweet potatoes,Yams,"Roots, Other",Sugar cane,Sugar beet,Sugar non-centrifugal,Sugar (Raw Equivalent),"Sweeteners, Other",Honey,Beans,Peas,"Pulses, Other and products",Nuts and products,Groundnuts (Shelled Eq),Sunflower seed,Rape and Mustardseed,Cottonseed,Coconuts - Incl Copra,Sesame seed,Palm kernels,Olives (including preserved),"Oilcrops, Other",Soyabeans,Soyabean Oil,Groundnut Oil,Sunflowerseed Oil,Rape and Mustard Oil,...,Palmkernel Oil,Coconut Oil,Sesameseed Oil,Olive Oil,Ricebran Oil,Maize Germ Oil,"Oilcrops Oil, Other",Palm Oil,Tomatoes and products,Onions,"Vegetables, Other",Pimento,"Oranges, Mandarines","Lemons, Limes and products",Grapefruit and products,"Citrus, Other",Bananas,Plantains,Apples and products,Pineapples and products,Dates,Grapes and products (excl wine),"Fruits, Other",Bovine Meat,Mutton & Goat Meat,Pigmeat,Poultry Meat,Eggs,Milk - Excluding Butter,Freshwater Fish,Demersal Fish,Pelagic Fish,"Marine Fish, Other",Crustaceans,Cephalopods,"Molluscs, Other","Meat, Aquatic Mammals","Aquatic Animals, Others",Aquatic Plants,Miscellaneous
5,5,AFG,162.517467,27.549035,0.865952,0.865952,0.865952,0.865952,0.865952,0.865952,6.333265,3.180261,3.180261,3.180261,3.180261,3.180261,2.514369,2.514369,2.514369,2.514369,2.514369,2.514369,16.666667,16.666667,16.666667,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,25.0,3.333333,3.333333,3.333333,3.333333,...,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,3.333333,6.8,75.0,75.0,75.0,75.0,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,18.181818,7.843679,6.156321,NaN,5.314043,1.846358,172.841561,3.111111,3.111111,3.111111,3.111111,3.111111,3.111111,3.111111,3.111111,3.111111,NaN,NaN


In [0]:
ctry_diet_df.head()

,Wheat and products,Rice (Milled Equivalent),Barley and products,Maize and products,Rye and products,Oats,Millet and products,Sorghum and products,"Cereals, Other",Cassava and products,Potatoes and products,Sweet potatoes,Yams,"Roots, Other",Sugar cane,Sugar beet,Sugar non-centrifugal,Sugar (Raw Equivalent),"Sweeteners, Other",Honey,Beans,Peas,"Pulses, Other and products",Nuts and products,Soyabeans,Groundnuts (Shelled Eq),Sunflower seed,Rape and Mustardseed,Cottonseed,Coconuts - Incl Copra,Sesame seed,Palm kernels,Olives (including preserved),"Oilcrops, Other",Soyabean Oil,Groundnut Oil,Sunflowerseed Oil,Rape and Mustard Oil,Cottonseed Oil,Palmkernel Oil,...,Dates,Grapes and products (excl wine),"Fruits, Other",Coffee and products,Cocoa Beans and products,Tea (including mate),Pepper,Pimento,Cloves,"Spices, Other",Wine,Beer,"Beverages, Fermented","Beverages, Alcoholic","Alcohol, Non-Food",Bovine Meat,Mutton & Goat Meat,Pigmeat,Poultry Meat,"Meat, Other","Offals, Edible","Butter, Ghee",Cream,"Fats, Animals, Raw","Fish, Body Oil","Fish, Liver Oil",Eggs,Milk - Excluding Butter,Freshwater Fish,Demersal Fish,Pelagic Fish,"Marine Fish, Other",Crustaceans,Cephalopods,"Molluscs, Other","Meat, Aquatic Mammals","Aquatic Animals, Others",Aquatic Plants,Infant food,Miscellaneous
Afghanistan,161.116667,14.810000,2.663333,2.540000,NaN,NaN,0.450000,NaN,0.000000,NaN,6.640000,NaN,NaN,NaN,NaN,NaN,NaN,8.326667,0.693333,0.070000,NaN,NaN,2.440000,2.076667,NaN,NaN,NaN,NaN,NaN,0.000000,0.546667,NaN,0.066667,NaN,0.540000,0.000000,0.616667,0.070000,0.11,NaN,...,0.173333,14.376667,7.333333,0.000000,0.020000,3.403333,0.000000,NaN,NaN,0.296667,0.000000,0.090000,NaN,0.000000,NaN,4.853333,5.210000,NaN,2.213333,0.390000,1.813333,1.166667,0.273333,0.520000,NaN,NaN,1.086667,61.123333,0.070000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063333,0.063333
Albania,138.976667,7.110000,0.160000,2.070000,0.676667,0.063333,NaN,NaN,0.073333,0.000000,40.453333,0.813333,NaN,0.00,NaN,NaN,NaN,17.443333,27.986667,0.943333,4.953333,0.000000,0.316667,3.966667,0.003333,0.133333,NaN,0.0,NaN,0.016667,NaN,NaN,20.946667,NaN,0.086667,0.010000,4.946667,0.016667,NaN,NaN,...,3.746667,47.260000,42.286667,2.376667,0.623333,0.090000,0.003333,0.003333,NaN,0.056667,7.666667,27.273333,0.000000,2.680000,NaN,22.046667,14.490000,10.983333,13.133333,0.040000,3.763333,1.120000,0.113333,1.696667,0.0,0.0,10.953333,301.613333,0.693333,0.790000,2.013333,0.730000,0.310000,0.380000,0.310000,NaN,0.0,0.0,0.076667,0.076667
Algeria,185.633333,2.610000,13.333333,15.530000,0.000000,0.003333,NaN,0.053333,0.023333,0.000000,64.936667,NaN,NaN,0.00,NaN,NaN,NaN,27.970000,0.583333,0.156667,1.663333,0.426667,5.330000,1.670000,0.000000,0.240000,NaN,0.0,NaN,0.136667,0.110000,0.0,3.726667,NaN,7.743333,0.473333,2.120000,0.436667,NaN,NaN,...,19.540000,11.636667,27.533333,3.266667,0.916667,0.390000,0.070000,0.480000,0.00,0.270000,0.010000,4.340000,0.000000,0.140000,NaN,5.426667,7.390000,0.000000,7.466667,0.360000,1.400000,0.526667,NaN,0.273333,0.0,0.0,7.316667,146.030000,0.246667,0.420000,2.710000,0.410000,0.113333,0.046667,0.010000,NaN,0.0,0.0,0.543333,0.543333
Angola,39.790000,8.040000,0.000000,38.396667,NaN,0.013333,1.476667,0.000000,NaN,197.060000,22.853333,41.766667,NaN,0.00,NaN,NaN,NaN,14.386667,0.010000,1.113333,8.956667,NaN,NaN,0.100000,0.400000,1.970000,NaN,NaN,NaN,0.170000,0.136667,NaN,NaN,NaN,3.603333,0.736667,0.220000,0.013333,0.03,0.33,...,0.000000,0.096667,2.410000,1.810000,0.166667,0.030000,0.000000,0.000000,0.00,0.013333,4.770000,54.126667,19.100000,1.986667,NaN,7.896667,1.080000,8.406667,16.470000,0.430000,1.120000,0.080000,NaN,0.633333,0.0,NaN,1.720000,11.973333,0.950000,6.313333,7.283333,1.173333,0.040000,0.060000,0.000000,NaN,0.0,0.0,0.193333,0.193333
Antigua and Barbuda,61.260000,9.046667,NaN,2.100000,0.133333,2.150000,NaN,NaN,0.880000,1.296667,15.236667,4.996667,1.343333,1.44,NaN,NaN,NaN,26.370000,4.660000,0.323333,1.420000,0.573333,0.696667,0.680000,0.160000,0.686667,0.05,0.0,NaN,0.580000,0.040000,NaN,0.

In [0]:
foods = list(dig_aa_df.columns)

In [0]:
# non eat lancet version
aa_total = {}

for country in ctry_list:
  #ctry_diet = ctry_diet_df2[ctry_diet_df2['index']==country]; # CHANGE BACK TO DF2
  #ctry_diet = ctry_diet_df[ctry_diet_df.index==country]
  ctry_diet = ctry_diet_el_df[ctry_diet_el_df['index']==country]
  aa_ctry = {}

  for AA in aa_list:
    aa_ctr = 0
    for food in foods:
      temp = ctry_diet[food].values[0] # kg/capita/yr
      temp2 = dig_aa_df.loc[AA,food] # g/100g

      aa = temp * temp2 * 10 / 365 # to correct for kg / 100g and for 365 d /yr
      if not math.isnan(aa):
        aa_ctr = aa_ctr + aa # adds the amount of the amino acid to tracker
        #aa_ctr.append(aa)

    aa_ctry[AA] = aa_ctr # stores the added amino acid content
    #print([AA, aa_ctr])
  aa_total[country] = aa_ctry

    
    

In [0]:
# Eat lancet version
aa_total = {}
ctry_list = list(ctry_diet_el_df['index'].unique()) # for EAT Lancet only

for country in ctry_list:
  #ctry_diet = ctry_diet_df2[ctry_diet_df2['index']==country]; # CHANGE BACK TO DF2
  #ctry_diet = ctry_diet_df[ctry_diet_df.index==country]
  ctry_diet = ctry_diet_el_df[ctry_diet_el_df['index']==country]
  aa_ctry = {}

  for AA in aa_list:
    aa_ctr = 0
    for food in foods:
      if food in ctry_diet.columns:
        temp = ctry_diet[food].values[0] # kg/capita/yr
        temp2 = dig_aa_df.loc[AA,food] # g/100g

      aa = temp * temp2 * 10 / 365 # to correct for kg / 100g and for 365 d /yr
      if not math.isnan(aa):
        aa_ctr = aa_ctr + aa # adds the amount of the amino acid to tracker
        #aa_ctr.append(aa)

    aa_ctry[AA] = aa_ctr # stores the added amino acid content
    #print([AA, aa_ctr])
  aa_total[country] = aa_ctry

    
    

In [0]:
aa_total

{'AFG': {'Alanine': 4.1928646679447645,
  'Arginine': 9.56875987834777,
  'Aspartic': 9.584620963065326,
  'Cystine': 1.80647514285888,
  'Glutamic': 24.08474134568552,
  'Glycine': 3.613967861039173,
  'Histidine': 3.7917094398115783,
  'Isoleucine': 5.9427244828782415,
  'Leucine': 10.658368034233883,
  'Lysine': 9.05410686460871,
  'Methionine': 2.956140608560545,
  'Phenylalanine': 6.479439321259715,
  'Serine': 4.768408907049912,
  'Threonine': 5.016133962000376,
  'Tryptophan': 1.6453330018630021,
  'Tyrosine': 4.635578685476192,
  'Valine': 6.702162301669956,
  'protein': 147.86601814014963},
 'AGO': {'Alanine': 3.4355347401963074,
  'Arginine': 13.828458438745226,
  'Aspartic': 8.476829814049212,
  'Cystine': 2.412567391194831,
  'Glutamic': 16.298195394042594,
  'Glycine': 2.7536709778758968,
  'Histidine': 5.64105432816924,
  'Isoleucine': 8.654175915589104,
  'Leucine': 15.154960000622829,
  'Lysine': 15.431871781221568,
  'Methionine': 4.811990147985573,
  'Phenylalanine': 

In [0]:
aa_total_df = pd.DataFrame.from_dict(aa_total,orient='index')
aa_total_df.head()

,Tryptophan,Threonine,Isoleucine,Leucine,Lysine,Methionine,Cystine,Phenylalanine,Tyrosine,Valine,Arginine,Histidine,Alanine,Aspartic,Glutamic,Glycine,Serine,protein
Afghanistan,0.487122,1.341323,1.672273,3.116425,2.068611,0.865279,0.660748,1.911089,1.270561,1.993538,2.282298,1.067132,1.213540,2.032668,9.877572,1.144424,1.585914,50.422024
Albania,0.913066,3.531826,4.052330,7.266889,6.539363,2.249113,0.927360,4.163924,2.951224,4.714799,4.280839,2.302172,1.645812,3.662816,11.448387,1.165103,2.165099,92.557332
Algeria,0.635470,2.136304,2.585817,4.720881,3.581070,1.372841,0.763208,2.854467,1.819949,3.063583,2.885787,1.432092,1.565148,3.113299,11.585099,1.299388,1.988267,67.967847
Angola,0.421027,1.545102,1.734780,3.232698,2.923258,1.012314,0.487358,1.780613,1.304326,1.969939,2.458712,1.162316,1.013359,1.825447,4.591620,0.723045,0.899647,42.747555
Antigua and Barbuda,0.909380,3.417069,3.779702,6.661607,6.811607,2.243767,0.933639,3.626478,2.805499,4.197814,4.906651,2.345322,1.674097,3.103161,7.741471,1.285173,1.629267,85.380345


In [0]:
savepath = path + 'diet_aa_1126_waste.csv'
aa_total_df.to_csv(savepath)

# Compare protein with FAO protein estimate

In [0]:
fao_data.head()# This is the FAO database to work with

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,Y1962F,Y1963,Y1963F,Y1964,Y1964F,Y1965,Y1965F,Y1966,Y1966F,Y1967,Y1967F,Y1968,Y1968F,Y1969,Y1969F,Y1970,Y1970F,Y1971,Y1971F,Y1972,Y1972F,Y1973,Y1973F,Y1974,Y1974F,Y1975,Y1975F,Y1976,Y1976F,Y1977,...,Y1994,Y1994F,Y1995,Y1995F,Y1996,Y1996F,Y1997,Y1997F,Y1998,Y1998F,Y1999,Y1999F,Y2000,Y2000F,Y2001,Y2001F,Y2002,Y2002F,Y2003,Y2003F,Y2004,Y2004F,Y2005,Y2005F,Y2006,Y2006F,Y2007,Y2007F,Y2008,Y2008F,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1000 persons,8954.00,NaN,9142.00,NaN,9340.00,NaN,9547.00,NaN,9765.00,NaN,9990.00,NaN,10222.00,NaN,10466.00,NaN,10729.00,NaN,11016.00,NaN,11323.00,NaN,11644.00,NaN,11966.00,NaN,12274.00,NaN,12552.00,NaN,12807.00,NaN,13034.00,...,16485.00,NaN,17586.00,NaN,18415.00,NaN,19021.00,NaN,19497.00,NaN,19987.00,NaN,20595.00,NaN,21348.00,NaN,22203.00,NaN,23116.00,NaN,24019.00,NaN,24861.00,NaN,25631.00,NaN,26349.00,NaN,27032.00,NaN,27708.00,NaN,28398.00,NaN,29105.00,NaN,29825.00,NaN,30552.00,NaN
1,2,Afghanistan,2901,Grand Total,664,Food supply (kcal/capita/day),kcal/capita/day,2999.00,Fc,2917.00,Fc,2698.00,Fc,2953.00,Fc,2956.00,Fc,2737.00,Fc,2971.00,Fc,2918.00,Fc,2935.00,Fc,2534.00,Fc,2512.00,Fc,2658.00,Fc,2721.00,Fc,2713.00,Fc,2752.00,Fc,2824.00,Fc,2489.00,...,1820.00,Fc,1844.00,Fc,1843.00,Fc,1874.00,Fc,1903.00,Fc,1852.00,Fc,1790.00,Fc,1737.00,Fc,1826.00,Fc,1892.00,Fc,1967.00,Fc,1948.00,Fc,1966.00,Fc,2046.00,Fc,2041.00,Fc,2081.00,Fc,2104.00,Fc,2107.00,Fc,2100.00,Fc,2090.00,Fc
2,2,Afghanistan,2901,Grand Total,674,Protein supply quantity (g/capita/day),g/capita/day,84.91,Fc,82.98,Fc,77.12,Fc,83.49,Fc,83.86,Fc,79.17,Fc,85.25,Fc,84.10,Fc,84.84,Fc,72.82,Fc,72.70,Fc,75.75,Fc,77.38,Fc,76.99,Fc,77.79,Fc,79.73,Fc,71.13,...,54.84,Fc,52.96,Fc,54.25,Fc,56.82,Fc,57.78,Fc,56.12,Fc,52.57,Fc,49.67,Fc,53.35,Fc,54.54,Fc,55.24,Fc,53.51,Fc,53.46,Fc,56.00,Fc,56.96,Fc,57.79,Fc,58.14,Fc,58.91,Fc,58.91,Fc,58.25,Fc
3,2,Afghanistan,2901,Grand Total,684,Fat supply quantity (g/capita/day),g/capita/day,37.51,Fc,37.61,Fc,38.57,Fc,38.95,Fc,39.73,Fc,39.95,Fc,41.85,Fc,41.99,Fc,41.50,Fc,37.92,Fc,35.18,Fc,34.64,Fc,37.20,Fc,38.81,Fc,39.95,Fc,41.95,Fc,38.40,...,31.88,Fc,40.66,Fc,38.86,Fc,34.24,Fc,35.57,Fc,38.02,Fc,32.14,Fc,26.96,Fc,29.95,Fc,29.99,Fc,34.95,Fc,36.75,Fc,31.13,Fc,32.09,Fc,29.72,Fc,30.72,Fc,33.88,Fc,33.08,Fc,33.37,Fc,33.52,Fc
4,2,Afghanistan,2903,Vegetal Products,664,Food supply (kcal/capita/day),kcal/capita/day,2752.00,Fc,2672.00,Fc,2438.00,Fc,2690.00,Fc,2682.00,Fc,2445.00,Fc,2666.00,Fc,2599.00,Fc,2623.00,Fc,2256.00,Fc,2253.00,Fc,2418.00,Fc,2466.00,Fc,2453.00,Fc,2491.00,Fc,2545.00,Fc,2221.00,...,1548.00,Fc,1557.00,Fc,1543.00,Fc,1559.00,Fc,1578.00,Fc,1511.00,Fc,1515.00,Fc,1535.00,Fc,1566.00,Fc,1654.00,Fc,1726.00,Fc,1715.00,Fc,1762.00,Fc,1839.00,Fc,1831.00,Fc,1871.00,Fc,1888.00,Fc,1891.00,Fc,1883.00,Fc,1873.00,Fc


In [0]:
# Want Element = "Protein supply quantity (g/capita/day)"
# and Item = 'Grand Total'

def country_protein(country):
  """Gets the food supply for all foods within a country for 2011-2013"""
  df_ctry = fao_data[fao_data['Area']==country]
  temp = df_ctry[(df_ctry['Element'] == 'Protein supply quantity (g/capita/day)') & 
                    (df_ctry['Item']=='Grand Total')][['Y2011','Y2012','Y2013']]
  ctry_protein = list(pd.DataFrame.mean(temp,axis=1)) # takes the average across years
  return ctry_protein

In [0]:
ctry_protein = {}
for country in ctry_list: # cycle through each country
  ctry_protein[country] = country_protein(country)
  

In [0]:
ctry_p_df = pd.DataFrame.from_dict(ctry_protein,orient='index')
ctry_p_df.head()

,0
Afghanistan,58.690000
Albania,110.440000
Algeria,90.743333
Angola,56.296667
Antigua and Barbuda,82.520000


In [0]:
savepath = path + 'fao_protein_2011-13.csv'
ctry_p_df.to_csv(savepath)